In [2]:
from nilearn import image, plotting
import numpy as np
import pandas as pd
import os
import glmsingle
from glmsingle.glmsingle import GLM_single
import time

In [2]:
input_directory='/zpool/vladlab/data_drive/fmri_tutorial' # The dataset to process
target_directory='/zpool/vladlab/data_drive/fmri_tutorial/derivatives' # The directory to store the preprocessed data
subject='spaceloc1001' # The subject to process

vols = 321 # Number of volumes in the fMRI data
tr = 1 # Repetition time in seconds
runs = 3 # Number of runs in the fMRI data

#load func file for run-01
func_file = f'{input_directory}/sub-{subject}/ses-01/func/sub-{subject}_ses-01_task-spaceloc_run-01_bold.nii.gz'
func_img = image.load_img(func_file)
stimdur = 20 # Duration of the stimulus in seconds



### Convert covariates into TR matched design matrix

In [10]:
conds = ['SA','FT'] # Conditions of the experiment
design_matrix = []
design_matrix_full = []
# loop through and load the condition files
for run in range(1,runs+1):
    run_design = pd.DataFrame(np.zeros((vols, len(conds))), columns=conds)
    run_design_full = pd.DataFrame(np.zeros((vols, len(conds))), columns =conds)
    for cond in conds:

        cov = pd.read_csv(f'{input_directory}/sub-{subject}/ses-01/func/SpaceLoc_spaceloc1001_Run{run}_{cond}.txt', sep='\t', header=None)
        cov.columns = ['onset', 'duration', 'value']
        # Downsample the covariate to match the TR
        cov['onset'] = (cov['onset'] / tr).astype(int)
        cov['duration'] = (cov['duration'] / tr).astype(int)

        #add a 1 to the design matrix for the onset of each condition
        run_design.loc[cov['onset'], cond] = 1

        #add a 1 to the design matrix for the onset of each condition and the duration
        for onset, duration in zip(cov['onset'], cov['duration']):
            run_design_full.loc[onset:onset+duration, cond] = 1

    #concatenate the design matrix for each run
    design_matrix.append(run_design.values)
    design_matrix_full.append(run_design_full.values)
  
   



### Load func images and concatenate them

In [5]:
img4d = []
for run in range(1, runs + 1):
    img = image.load_img(f'{input_directory}/sub-{subject}/ses-01/func/sub-{subject}_ses-01_task-spaceloc_run-0{run}_bold.nii.gz')
    img4d.append(img.get_fdata())

# Stack the 3D images into a 4D image
#img4d_data = np.stack(img4d, axis=-1)
print(f'img4d_data shape: {img4d_data.shape}')

img4d_data shape: (64, 64, 48, 321, 3)


In [5]:
opt = dict()

# set important fields for completeness (but these would be enabled by default)
# wantlibrary = 1 -> fit HRF to each voxel 
# wantglmdenoise = 1 -> use GLMdenoise 
# wantfracridge = 1 -> use ridge regression to improve beta estimates 
opt['wantlibrary'] = 1
opt['wantglmdenoise'] = 1
opt['wantfracridge'] = 1

# for the purpose of this example we will keep the relevant outputs in memory
# and also save them to the disk
opt['wantfileoutputs'] = [1,1,1]
opt['wantmemoryoutputs'] = [0,0,0]
glmsingle_obj = GLM_single(opt)

In [8]:
glmsingle_obj = GLM_single()

In [ ]:
#create directory for the subject output
os.makedirs(f'{target_directory}/sub-{subject}/ses-01/glmsingle/', exist_ok=True)

#might need to change number of CPUs before running: export OPENBLAS_NUM_THREADS=4


start_time = time.time()
results_glmsingle = glmsingle_obj.fit(
    design_matrix,
    img4d,
    stimdur,
    tr,
    outputdir=f'{target_directory}/sub-{subject}/ses-01/glmsingle/')

elapsed_time = time.time() - start_time
print(f'GLM_single processing time: {elapsed_time:.2f} seconds')


*** DIAGNOSTICS ***:
There are 3 runs.
The number of conditions in this experiment is 2.
The stimulus duration corresponding to each trial is 20.00 seconds.
The TR (time between successive data points) is 1.00 seconds.
The number of trials in each run is: [10, 10, 10].
The number of trials for each condition is: [np.int64(15), np.int64(15)].
For each condition, the number of runs in which it appears: [np.int64(3), np.int64(3)].
For each run, how much ending buffer do we have in seconds? [np.int64(30), np.int64(30), np.int64(50)].
*** Saving design-related results to /zpool/vladlab/data_drive/fmri_tutorial/derivatives/sub-spaceloc1001/ses-01/glmsingle/DESIGNINFO.npy. ***
*** FITTING DIAGNOSTIC RUN-WISE FIR MODEL ***
*** Saving FIR results to /zpool/vladlab/data_drive/fmri_tutorial/derivatives/sub-spaceloc1001/ses-01/glmsingle/RUNWISEFIR.npy. ***

*** FITTING TYPE-A MODEL (ONOFF) ***

fitting model...
done.

preparing output...
done.

computing model fits...
done.

computing R^2...


: 

In [1]:
np.show_config()

NameError: name 'np' is not defined

In [ ]:
#create directory for the subject output
os.makedirs(f'{target_directory}/sub-{subject}/ses-01/glmsingle/design_full', exist_ok=True)


start_time = time.time()
results_glmsingle = glmsingle_obj.fit(
    design_matrix_full.values,
    img4d_data,
    stimdur,
    tr,
    outputdir=f'{target_directory}/sub-{subject}/ses-01/glmsingle/design_full')

elapsed_time = time.time() - start_time
print(f'GLM_single processing time: {elapsed_time:.2f} seconds')


In [17]:
#load cov files

cov_file = pd.read_csv(f'{input_directory}/sub-{subject}/ses-01/func/SpaceLoc_spaceloc1001_Run1_SA.txt', sep='\t', header=None)
cov_file.columns = ['onset', 'duration', 'value']

# Downsample the covariate to match the TR
cov_file['onset'] = (cov_file['onset'] / tr).astype(int)
cov_file['duration'] = (cov_file['duration'] / tr).astype(int)


In [19]:
#create a design matrix of size (vols)
design_matrix = np.zeros((vols, 1))

design_matrix[cov_file['onset'], 0] = 1  # Set the onset of the stimulus to 1


In [ ]:
#timing file 

In [9]:
func_img.shape

(64, 64, 48, 321)

In [ ]:
vols = 321
tr = 1

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : np.bytes_(b'')
db_name         : np.bytes_(b'')
extents         : 0
session_error   : 0
regular         : np.bytes_(b'r')
dim_info        : 57
dim             : [  4  64  64  48 321   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : uint16
bitpix          : 16
slice_start     : 0
pixdim          : [-1.  3.  3.  3.  1.  0.  0.  0.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : np.bytes_(b'TE=30;Time=170354.233;phase=1;mb=4')
aux_file        : np.bytes_(b'Unaliased_MB4_PE3_LB')
qform_code      : scanner
sform_code      : scanner
quatern_b       : 0.03546662
quatern_c       : 0.9993094
quatern_d  